In [1]:
%load_ext jupyter_black

In [41]:
import distributed
from dask_cuda import LocalCUDACluster

cluster = LocalCUDACluster()
client = distributed.Client(cluster)
client

/opt/conda/envs/rapids/lib/python3.9/site-packages/dask_cuda/utils.py:244: UserWarning: Cannot get CPU affinity for device with index 0, setting default affinity
  warnings.warn(
2022-08-08 00:10:52,503 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-fola5982', purging
2022-08-08 00:10:52,504 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-52ey3c8b', purging
2022-08-08 00:10:52,504 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-oxyqabo7', purging
2022-08-08 00:10:52,505 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2022-08-08 00:10:52,505 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize


Connection method: Cluster object,Cluster type: dask_cuda.LocalCUDACluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 1
Total threads: 1,Total memory: 15.57 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:37221,Workers: 1
Dashboard: http://127.0.0.1:8787/status,Total threads: 1
Started: Just now,Total memory: 15.57 GiB
Comm: tcp://127.0.0.1:35839,Total threads: 1
Dashboard: http://127.0.0.1:37199/status,Memory: 15.57 GiB
Nanny: tcp://127.0.0.1:43937,


In [8]:
import os
import uuid
from pathlib import Path
from warnings import warn
from datetime import datetime
from typing import Callable, Union, TypedDict
import cupy
import cudf
from requests import Session, HTTPError

import pandas as pd
import numpy as np
import dask.dataframe as dd
from dask.distributed import Client, LocalCluster
from geopandas import GeoDataFrame
from requests import Session, HTTPError

PROBSEVERE_URL_TEMPLATE = (
    "https://mtarchive.geol.iastate.edu/%Y/%m/%d/mrms/ncep/ProbSevere/MRMS_PROBSEVERE_%Y%m%d_%H%M00.json"
)
TimeLike = Union[datetime, str, pd.Timestamp]

archives = Path(os.getcwd().split("/notebooks")[0]) / "archive"

FILE_OUT_DIR = os.path.abspath("../../bucket2")

TimeLike = Union[datetime, str, pd.Timestamp]


class Feature(TypedDict):
    properties: dict[str, Union[float, int, str]]


class FeatureCollection(TypedDict):
    validTime: str
    features: list[Feature]

In [49]:
__uuid = uuid.uuid4()


def __batch_id(validtime: datetime) -> Callable[[int], str]:
    def wrapper(n: int) -> str:
        return f"probsevere-{n}-{validtime.isoformat(timespec='minutes')}-{__uuid}.parquet"

    return wrapper


def __features(features: list[Feature]) -> pd.DataFrame:
    return GeoDataFrame.from_features(features)


def __bounds(
    df: pd.DataFrame,
    drop_columns: list[str] = ["MAXRC_EMISS", "MAXRC_ICECF", "AVG_BEAM_HGT", "geometry"],
) -> pd.DataFrame:
    return pd.concat((df, df["geometry"].bounds), axis=1).drop(columns=drop_columns)


def direct_to_parquet(
    path_dir: Path,
    start: TimeLike,
    end: TimeLike,
    freq: str = "2min",
) -> None:
    """scrape data from the iastate archive over a daterange"""
    # create a DatetimeIndex using the the function arguments and format the urls using the url template
    urls = pd.date_range(start=start, end=end, freq=freq)
    # using Session as a context manager
    if not path_dir.exists():
        path_dir.mkdir()
    with Session() as session:
        # iterating over all of the urls
        for url in urls.strftime(PROBSEVERE_URL_TEMPLATE):
            # using a try/catch block in the event the download fails
            try:
                # with our session make a get request, r is a response object
                r = session.get(url, stream=True)
                # in the event of a non 200 status code we'll raise a HTTPError and trigger the except block
                r.raise_for_status()
            # if there was an error downloading, continue
            except (ConnectionError, HTTPError):
                warn(f"error downloading {url}")
                continue
            fc: FeatureCollection = r.json()

            features = fc["features"]
            # in the event no storms were record, continue
            if not features:
                continue
            #
            df = __features(features).pipe(__bounds).astype(np.float32)

            valid_time = datetime.strptime(fc["validTime"], "%Y%m%d_%H%M%S %Z")
            # set the valid time
            df["valid_time"] = valid_time.timestamp()
            print("yerp")
            # cupy.from_nun
            # parquet.to_parquet
            # print(path_dir / valid_time.strftime("%Y-%m-%dT%H:%M.pq"))
            cudf.from_pandas(df).to_parquet(
                path_dir / valid_time.strftime("%Y-%m-%dT%H:%M.pq"),
                # append=True,
                # partition_file_name=__batch_id(valid_time),
                # ignore_divisions=True,
            )

In [50]:
if __name__ == "__main__":
    # NOTE: the first time this ran it collected files from 2022-03-01T00:00:00Z -> 2022-05-14T15:54:00Z
    direct_to_parquet(
        Path(FILE_OUT_DIR),
        # start="2022-03-01T00:00:00Z",
        start="2022-07-15T00:00",
        end="2022-07-15T00:02",
    )

yerp
yerp


In [38]:
import os
from pathlib import Path
import cudf
import dask.dataframe as dd

files = tuple(Path(os.path.abspath("../../bucket")).rglob("*.parquet"))

set_1, set_2 = files[:41260], files[41260:]

In [39]:
df1:cudf.DataFrame = cudf.read_parquet(set_1)
df1

,MUCAPE,MLCAPE,MLCIN,EBSHEAR,SRH01KM,MEANWIND_1-3kmAGL,MESH,VIL_DENSITY,FLASH_RATE,FLASH_DENSITY,...,SIZE,MOTION_EAST,MOTION_SOUTH,PS,ID,minx,miny,maxx,maxy,valid_time
__null_dask_index__,,,,,,,,,,,,,,,,,,,,,
0,1159.0,967.0,0.0,54.700001,87.0,20.700001,1.24,3.19,24.0,0.54,...,200.0,8.789,-4.669,92.0,181334.0,-80.800003,28.690001,-80.650002,28.850000,2022-03-16 05:00:38
1,1642.0,1403.0,0.0,32.200001,38.0,16.700001,0.07,0.82,0.0,0.04,...,355.0,2.935,-6.017,3.0,181727.0,-79.910004,26.820000,-79.669998,27.080000,2022-03-16 05:00:38
2,489.0,377.0,-2.0,16.100000,89.0,24.200001,0.00,0.60,0.0,0.03,...,266.0,1.097,-5.537,0.0,181916.0,-87.050003,33.450001,-86.879997,33.770000,2022-03-16 05:00:38
3,916.0,691.0,0.0,18.200001,56.0,14.900000,0.28,1.47,2.0,0.18,...,304.0,-0.587,-5.366,1.0,181995.0,-87.580002,32.299999,-87.320000,32.520000,2022-03-16 05:00:38
4,1166.0,688.0,-5.0,44.500000,112.0,24.200001,0.26,1.31,6.0,0.28,...,242.0,10.067,-0.850,11.0,182018.0,-81.379997,28.090000,-81.230003,28.330000,2022-03-16 05:00:38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,2732.0,2102.0,-7.0,27.100000,23.0,6.400000,0.23,0.88,3.0,0.14,...,83.0,6.669,0.398,4.0,213213.0,-93.080002,32.580002,-92.989998,32.669998,2022-06-05 23:52:41
150,2756.0,2162.0,-8.0,17.900000,8.0,3.300000,0.10,0.65,1.0,0.05,...,40.0,3.134,-3.634,1.0,213214.0,-86.690002,32.470001,-86.639999,32.540001,2022-06-05 23:52:41
151,3154.0,1268.0,-10.0,27.600000,0.0,1.600000,0.00,0.67,0.0,0.00,...,79.0,2.360,-0.621,2.0,213215.0,-81.239998,26.110001,-81.139999,26.209999,2022-06-05 23:52:41


In [40]:
df2:cudf.DataFrame = cudf.read_parquet(set_2)
df2

,MUCAPE,MLCAPE,MLCIN,EBSHEAR,SRH01KM,MEANWIND_1-3kmAGL,MESH,VIL_DENSITY,FLASH_RATE,FLASH_DENSITY,...,SIZE,MOTION_EAST,MOTION_SOUTH,PS,ID,minx,miny,maxx,maxy,valid_time
__null_dask_index__,,,,,,,,,,,,,,,,,,,,,
0,1086.0,927.0,-3.0,47.400002,125.0,47.500000,0.00,0.60,1.0,0.05,...,466.0,11.851,-12.212,5.0,34389.0,-96.680000,31.770000,-96.419998,32.150002,2022-03-30 10:44:42
1,627.0,542.0,-5.0,37.099998,203.0,54.400002,0.00,0.73,0.0,0.05,...,62.0,11.909,-13.154,2.0,34672.0,-93.949997,35.950001,-93.870003,36.029999,2022-03-30 10:44:42
2,121.0,0.0,-999.0,20.000000,424.0,32.500000,0.00,0.69,2.0,0.03,...,549.0,8.522,-8.309,0.0,34782.0,-87.919998,44.220001,-87.510002,44.490002,2022-03-30 10:44:42
3,373.0,302.0,-10.0,27.799999,199.0,50.700001,0.05,0.88,1.0,0.09,...,369.0,9.868,-9.820,1.0,34787.0,-93.320000,36.990002,-93.180000,37.410000,2022-03-30 10:44:42
4,76.0,0.0,-999.0,28.100000,0.0,18.299999,0.00,0.55,0.0,0.00,...,402.0,9.179,5.616,1.0,34871.0,-103.610001,34.400002,-103.300003,34.619999,2022-03-30 10:44:42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,1261.0,813.0,-6.0,20.799999,90.0,24.100000,0.00,0.25,1.0,0.05,...,50.0,6.193,-4.321,0.0,126248.0,-79.059998,32.049999,-79.000000,32.119999,2022-04-18 14:50:38
60,2594.0,1942.0,-999.0,14.100000,39.0,28.000000,0.26,0.94,3.0,0.12,...,75.0,6.532,0.406,1.0,126249.0,-79.599998,30.879999,-79.519997,30.980000,2022-04-18 14:50:38
61,2491.0,2033.0,-999.0,14.800000,45.0,25.400000,0.11,0.45,0.0,0.05,...,114.0,6.470,-4.145,1.0,126250.0,-79.449997,30.299999,-79.300003,30.420000,2022-04-18 14:50:38


In [56]:
df0:cudf.DataFrame = cudf.concat((df1,df2))#.set_index(["valid_time","ID"])
rows,cols = df0.shape
rows,cols, rows*cols
# import numpy as np
# np.multiply(*df0.shape)

(5979869, 27, 161456463)

In [57]:
df0.set_index(["valid_time","ID"])

MUCAPE  MLCAPE  MLCIN    EBSHEAR  SRH01KM  \
valid_time          ID                                                    
2022-03-16 05:00:38 181334.0  1159.0   967.0    0.0  54.700001     87.0   
                    181727.0  1642.0  1403.0    0.0  32.200001     38.0   
                    181916.0   489.0   377.0   -2.0  16.100000     89.0   
                    181995.0   916.0   691.0    0.0  18.200001     56.0   
                    182018.0  1166.0   688.0   -5.0  44.500000    112.0   
...                              ...     ...    ...        ...      ...   
2022-04-18 14:50:38 126248.0  1261.0   813.0   -6.0  20.799999     90.0   
                    126249.0  2594.0  1942.0 -999.0  14.100000     39.0   
                    126250.0  2491.0  2033.0 -999.0  14.800000     45.0   
                    126251.0  3186.0  1898.0   -3.0  27.600000     41.0   
                    126252.0  2575.0  2067.0    0.0  21.600000     13.0   

                              MEANWIND_1-3kmAGL  MESH  VIL_DENSITY  \
valid_time          ID                                               
2022-03-16 05:00:38 181334.0          20.700001  1.24         3.19   
                    181727.0          16.700001  0.07         0.82   
                    181916.0          24.200001  0.00         0.60   
                    181995.0          14.900000  0.28         1.47   
                    182018.0          24.200001  0.26         1.31   
...                                         ...   ...          ...   
2022-04-18 14:50:38 126248.0          24.100000  0.00         0.25   
                    126249.0          28.000000  0.26         0.94   
                    126250.0          25.400000  0.11         0.45   
                    126251.0          10.400000  0.00         0.64   
                    126252.0           4.500000  0.00         0.47   

                              FLASH_RATE  FLASH_DENSITY  ...  CAPE_M10M30  \
valid_time          ID                                   ...                
2022-03-16 05:00:38 181334.0        24.0           0.54  ...        253.0   
                    181727.0         0.0           0.04  ...        490.0   
                    181916.0         0.0           0.03  ...        128.0   
                    181995.0         2.0           0.18  ...        321.0   
                    182018.0         6.0           0.28  ...        260.0   
...                                  ...            ...  ...          ...   
2022-04-18 14:50:38 126248.0         1.0           0.05  ...        213.0   
                    126249.0         3.0           0.12  ...        470.0   
                    126250.0         0.0           0.05  ...        537.0   
                    126251.0         0.0           0.00  ...        606.0   
                    126252.0         0.0           0.00  ...        630.0   

                              LJA   SIZE  MOTION_EAST  MOTION_SOUTH    PS  \
valid_time          ID                                                      
2022-03-16 05:00:38 181334.0  2.6  200.0        8.789        -4.669  92.0   
                    181727.0  0.0  355.0        2.935        -6.017   3.0   
                    181916.0  0.0  266.0        1.097        -5.537   0.0   
                    181995.0  0.0  304.0       -0.587        -5.366   1.0   
                    182018.0  0.0  242.0       10.067        -0.850  11.0   
...                           ...    ...          ...           ...   ...   
2022-04-18 14:50:38 126248.0  0.0   50.0        6.193        -4.321   0.0   
                    126249.0  0.0   75.0        6.532         0.406   1.0   
                    126250.0  0.0  114.0        6.470        -4.145   1.0   
                    126251.0  0.0   72.0        0.669        -5.493   2.0   
                    126252.0  0.0   48.0       -1.329        -3.748   1.0   

                                   minx       miny       maxx       maxy  
valid_time          ID                                          